In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/volleyball

/content/drive/My Drive/volleyball


In [ ]:
!git clone --depth 1 https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies


fatal: destination path 'yolov5' already exists and is not an empty directory.


In [6]:
import torch
from IPython.display import Image  # for displaying images
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 2.0.1+cu118 CPU


In [7]:
%cat ./yolov5/data/data.yaml

train: ./train
val: ./valid
test: ./test

nc: 1
names: ['Volleyball']

In [10]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [11]:
# data.yaml contains the information about number of classes and their labels required for this project
import yaml
with open("./yolov5/data/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [12]:
#this is the model configuration
# yolov5l.yaml contains the configuration of neural network required for training.
%cat /content/drive/MyDrive/volleyball/yolov5/models/yolov5x.yaml

# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, 

In [13]:
%%writetemplate /content/drive/MyDrive/volleyball/yolov5/models/yolov5x.yaml
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## Train Custom YOLOv5 Detector

### Next, we'll fire off training!¶
Here, we are able to pass a number of arguments:

* **img:** define input image size
* **batch:** determine batch size
* **epochs:** define the number of training epochs.
* **data:** set the path to our yaml file
* **cfg:** specify our model configuration
* **weights:** specify a custom path to weights.
* **name:** result names
* **nosave:** only save the final checkpoint
* **cache:** cache images for faster training


In [14]:
import os
os.chdir('/content/drive/MyDrive/volleyball/yolov5')


In [ ]:
%%time
%cd /content/drive/MyDrive/volleyball/yolov5
!python train.py --img-size 640  --batch 16 --epochs 50 --data './data/data.yaml' --cfg ./models/yolov5x.yaml --weights './runs/train/yolov5x_results3/weights/best.pt'  --name yolov5x_results  --cache

/content/drive/MyDrive/volleyball/yolov5
train: weights=./runs/train/yolov5x_results3/weights/best.pt, cfg=./models/yolov5x.yaml, data=./data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5x_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 bb9706e Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, war

In [ ]:
# Start tensorboard
# Launch after you have started training to all the graphs needed for inspection
# logs save in the folder "runs"
%reload_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/volleyball/yolov5/runs/train/yolov5x_results6

In [16]:
# use the best weights!
# Final weights will be by-default stored at /content/drive/MyDrive/yolov5_ex/yolov5/runs/train/yolov5l_results/weights/best.pt
%cd /content/drive/MyDrive/volleyball/yolov5
!python detect.py --weights /content/drive/MyDrive/volleyball/yolov5/runs/train/yolov5x_results6/weights/best.pt --img-size 640   --conf 0.4 --source ./vol.mp4

/content/drive/MyDrive/volleyball/yolov5
detect: weights=['/content/drive/MyDrive/volleyball/yolov5/runs/train/yolov5x_results6/weights/best.pt'], source=./vol.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 bb9706e Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x summary: 286 layers, 46600566 parameters, 0 gradients
video 1/1 (1/3497) /content/drive/MyDrive/volleyball/yolov5/vol.mp4: 384x640 2 Volleyballs, 59.7ms
video 1/1 (2/3497) /content/drive/MyDrive/volleyball/yolov5/vol.mp4: 384x640 2 Volleyballs, 33.9ms
video 1/1 (3/3497) /content/drive/MyDrive/volleyball/yolov5/vol.mp4:

In [ ]:
%cd /content/drive/MyDrive/volleyball/yolov5
!python detect.py --weights /content/drive/MyDrive/volleyball/yolov5/runs/train/yolov5x_results6/weights/best.pt --img 640  --conf 0.4 --source ./my_video.mp4

/content/drive/MyDrive/volleyball/yolov5
detect: weights=['/content/drive/MyDrive/volleyball/yolov5/runs/train/yolov5x_results6/weights/best.pt'], source=./my_video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 bb9706e Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5x summary: 286 layers, 46600566 parameters, 0 gradients
video 1/1 (1/4788) /content/drive/MyDrive/volleyball/yolov5/my_video.mp4: 384x640 2 Volleyballs, 50.1ms
video 1/1 (2/4788) /content/drive/MyDrive/volleyball/yolov5/my_video.mp4: 384x640 3 Volleyballs, 33.8ms
video 1/1 (3/4788) /content/drive/MyDrive/volleyball/

In [ ]:
pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt


In [16]:
%cd /content/drive/MyDrive/volleyball/yolov5
import cv2
import time
import torch
from pathlib import Path

# YOLOv5 modelini yüklemek için gereken modülü içe aktarın
from models.yolo import Model

# Modeli yükleyin
modelpath = "./runs/train/yolov5x_results6/weights/best.pt"
model = Model(modelpath)  # Modeli yükleyin

# Video dosyasının adını ve kaydedilecek dizini belirleyin
video_file = "my_video.mp4"
output_directory = "./result"

# output_directory'i oluşturun
Path(output_directory).mkdir(parents=True, exist_ok=True)

# Videoyu açın
cap = cv2.VideoCapture(video_file)

# CSV raporu oluşturmak için başlık satırını yazın
with open("output.csv", "w") as csv_file:
    csv_file.write("Zaman|Ekran Görüntüsü Adı\n")

# Başlangıç zamanını al
start_time = time.time()

# Videonun her bir frame'ini işle
frame_number = 0

# Videoyu işle
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1

    # Modeli kullanarak voleybol topunu tespit et
    results = model(frame)

    # Voleybol topunu tespit edilen kareleri işle
    for det in results.pred[0]:
        class_id = int(det[5])
        class_name = model.names[class_id]
        confidence = float(det[4])
        bbox = det[:4].tolist()

        if class_name == "Volleyball" and confidence >= 0.7:  # Sadece güvenilir tahminleri alın
            elapsed_time = time.time() - start_time
            timestamp = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
            screenshot_name = f"result_{int(time.time())}.jpg"
            screenshot_path = f"{output_directory}/{screenshot_name}"

            # Ekran görüntüsünü kaydet
            cv2.imwrite(screenshot_path, frame)

            # CSV raporuna bilgileri ekleyin
            with open("output.csv", "a") as csv_file:
                csv_file.write(f"{timestamp}|{screenshot_name}\n")

# Temizlik
cap.release()
cv2.destroyAllWindows()

# CSV dosyasını kapat
csv_file.close()


/content/drive/My Drive/volleyball


ReaderError: ignored